# Read email, collect attachments, merge rows

pip install keyring

In [ ]:
pip install imbox

In [ ]:
pip install python-docx

In [ ]:
pip install openpyxl

In [ ]:
pip install yagmail

In [ ]:
pip install pandas

In [ ]:
pip install jinja2

In [87]:
from imbox import Imbox
from docx import Document
from openpyxl import load_workbook, Workbook
import os
import glob
from datetime import datetime
import yagmail
import pandas as pd
from openpyxl.styles import PatternFill

def makesurePath(path):
    if not os.path.exists(path):
        os.mkdir(path)

def getAttachmentPathFromMessage(message, saveFolderName):
    subject = message.subject
    formateDate = message.date.replace(' ', '-').replace(':', '-')
    emailFolderName = formateDate + '_' + subject
    # saveFolderName2 = savePath + saveFolderName + '\\' + emailFolderName
    saveFolderName2 = savePath + saveFolderName
    return saveFolderName2

def readEmail(imapServer, myEmailName, myPassword, savePath, saveFolderName, titleKeyword):
    matchedMessages = []
    
    makesurePath(savePath + saveFolderName)

    with Imbox(imapServer, myEmailName, myPassword) as imbox:
        all_inbox_messages = imbox.messages()
        for uid, message in all_inbox_messages:
            lengthKeyword = len(titleKeyword)
            if message.subject[:lengthKeyword] == titleKeyword:
                matchedMessages.append(message)
                if message.attachments:
                    for attachment in message.attachments:
                        saveFolderName2 = getAttachmentPathFromMessage(message, saveFolderName)
                        makesurePath(saveFolderName2)
                        attachmentFilePath = saveFolderName2 + fr'\{attachment["filename"]}'
                        with open(attachmentFilePath, 'wb') as file:
                            file.write(attachment['content'].getvalue())

        return matchedMessages
    
def sendEmail(smtpServer, myEmailName, myPassword, targetEmailName, targetEmailTitle, targetEmailBody):
    #return
    mail = yagmail.SMTP(user=myEmailName, password=myPassword, host=smtpServer)
    contents = [targetEmailBody]
    mail.send(targetEmailName, targetEmailTitle, contents)

def mergeExcels(excels, saveFileMerged, saveFileMergedNoduplicate):
    print('mergeExcels ing, files count:' + str(len(excels)))
    destination_workbook = Workbook()
    destination_sheet = destination_workbook.active
    destination_sheet.append(['序号','姓名','学号'])

    for path in excels:
        if not path.endswith('.xlsx'):
            return
        for file in glob.glob(path):
            workbook = load_workbook(file)
            source_sheet = workbook.active
            for table_row in range(2, source_sheet.max_row + 1):
                row_data = [source_sheet.cell(row=table_row, column=col).value for col in range(1, source_sheet.max_column + 1)]
                destination_sheet.append(row_data)
    
    destination_workbook.save(saveFileMerged)

    df = pd.read_excel(saveFileMerged)
    duplicates = df["学号"].duplicated()
    duplicate_line_numbers =  [value + 2 for value in df[duplicates].index.tolist()]
    markExcel(saveFileMerged, duplicate_line_numbers, saveFileMergedNoduplicate)

def markExcel(excelFilePath, lineNumbers, saveFileMergedNoduplicate):
    wb = load_workbook(excelFilePath)
    ws = wb.active

    for row_number in lineNumbers:
        for col_number, cell in enumerate(ws[row_number], start=1):
            cell.fill = PatternFill(start_color="FF0000", end_color="FF0000", fill_type="solid")

    wb.save(saveFileMergedNoduplicate)

if __name__ == '__main__':
    # Set the path to your file path
    os.chdir('/')
    imapServer = r'imap.126.com'
    smtpServer = r'smtp.126.com'
    savePath = r'C:\Users\yourname\Downloads\2023-11-16'
    saveFolderName = r'\attachments'
    titleKeyword = 'Your Title keywords'

    myEmailName = 'example@126.com'
    myPassword = 'SSNQNGZSZKYSFTSX'

    targetEmailTitle = '[Reply]Thank you for apply'
    targetEmailBody = 'We are received your apply email.'

    mergeExcelsPath = savePath + fr'\merged.xlsx'
    mergeExcelsNoDuplicatePath = savePath + fr'\mergedNoDuplicate.xlsx'

    matchedMessages = readEmail(imapServer, myEmailName, myPassword, savePath, saveFolderName, titleKeyword)
    excels = []
    for message in matchedMessages:
        for attachment in message.attachments:
            if(attachment["filename"].endswith('.xlsx')):
                saveFolderName2 = getAttachmentPathFromMessage(message, saveFolderName)
                attachmentFilePath = saveFolderName2 + fr'\{attachment["filename"]}'
                excels.append(attachmentFilePath)
        responseEmail = message.sent_from[0]["email"]
        sendEmail(smtpServer, myEmailName, myPassword, responseEmail, targetEmailTitle, targetEmailBody)
    mergeExcels(excels, mergeExcelsPath, mergeExcelsNoDuplicatePath)
    print('done')


mergeExcels ing, files count:6
done
